<a href="https://colab.research.google.com/github/subbu27iitb/Anomaly-Detection/blob/main/Isolation_Forest_Anamoly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reference**

https://www.youtube.com/watch?v=DoUhCLbLeH0

https://grabngoinfo.com/isolation-forest-for-anomaly-detection-and-imbalanced-classification/

In [2]:
#Synthetic dataset
from sklearn.datasets import make_classification
#Data processing
import pandas as pd
import numpy as np
from collections import Counter
# Model and performance
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Create Imbalanced Dataser

In [29]:
# Create an imbalanced dataset
X, y = make_classification(n_samples=100000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=2,
                           n_clusters_per_class=1,
                           weights=[0.995, 0.005],
                           class_sep=0.5, random_state=0)
# Convert the data from numpy array to a pandas dataframe
df = pd.DataFrame({'feature1': X[:, 0], 'feature2': X[:, 1], 'target': y})
# Check the target distribution
df['target'].value_counts(normalize = True)


0    0.9897
1    0.0103
Name: target, dtype: float64

In [30]:
   #check
df.head()

,feature1,feature2,target
0,1.236497,0.579272,0
1,-0.188720,-0.022001,0
2,-0.653844,0.794309,0
3,0.005572,0.593370,0
4,0.482573,0.796902,0


In [31]:
  #Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 #Check the number of records
print('The number of records in the training dataset is', X_train.shape[0])
print('The number of records in the test dataset is', X_test.shape[0])
print(f"The training dataset has {sorted(Counter(y_train).items())[0][1]} records for the majority class and {sorted(Counter(y_train).items())[1][1]} records for the minority class.")

The number of records in the training dataset is 80000
The number of records in the test dataset is 20000
The training dataset has 79183 records for the majority class and 817 records for the minority class.


In [32]:
 #check
#Counter(y_train)
Counter(y_train).items()

dict_items([(0, 79183), (1, 817)])

In [33]:
# Train the isolation forest model
if_model = IsolationForest(n_estimators=100, random_state=0).fit(X_train)
# Predict the anomalies
if_prediction = if_model.predict(X_test)
# Change the anomalies' values to make it consistent with the true values
if_prediction = [1 if i==-1 else 0 for i in if_prediction]
# Check the model performance
print(classification_report(y_test, if_prediction))

              precision    recall  f1-score   support

           0       0.99      0.79      0.88     19787
           1       0.02      0.40      0.04       213

    accuracy                           0.78     20000
   macro avg       0.51      0.59      0.46     20000
weighted avg       0.98      0.78      0.87     20000



Isolation Forest with warm start on New data

In [36]:
# Create more imbalanced data
X_more, y_more = make_classification(n_samples=5000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=2,
                           n_clusters_per_class=1,
                           weights=[0.995, 0.005],
                           class_sep=0.5, random_state=0)

In [37]:
# Train the isolation forest model
if_model = IsolationForest(n_estimators=100, random_state=0, warm_start=True).fit(X_train)
# Use new data to train 50 trees on top of existing model 
if_model.n_estimators += 50
if_model.fit(X_more)
# Predict the anomalies
if_prediction = if_model.predict(X_test)
# Change the anomalies' values to make it consistent with the true values
if_prediction = [1 if i==-1 else 0 for i in if_prediction]
# Check the model performance
print(classification_report(y_test, if_prediction))

              precision    recall  f1-score   support

           0       0.99      0.73      0.84     19787
           1       0.02      0.40      0.03       213

    accuracy                           0.73     20000
   macro avg       0.50      0.57      0.44     20000
weighted avg       0.98      0.73      0.83     20000



 Isolation Forest With Warm Start On New Trees

In [38]:
# Train the isolation forest model
if_model = IsolationForest(n_estimators=100, random_state=0, warm_start=True).fit(X_train)
# Use the existing data to train 20 trees on top of existing model 
if_model.n_estimators += 20
if_model.fit(X_train)
# Predict the anomalies
if_prediction = if_model.predict(X_test)
# Change the anomalies' values to make it consistent with the true values
if_prediction = [1 if i==-1 else 0 for i in if_prediction]
# Check the model performance
print(classification_report(y_test, if_prediction))

              precision    recall  f1-score   support

           0       0.99      0.79      0.88     19787
           1       0.02      0.39      0.04       213

    accuracy                           0.79     20000
   macro avg       0.51      0.59      0.46     20000
weighted avg       0.98      0.79      0.87     20000

